# Storks class-aware

In [1]:
from utils import *

import os

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import pandas as pd
import numpy as np

from sklearn import preprocessing

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# Misc
LOG_DIR = './log_pt/'
MODEL_ID = 'storkes_plain'

# Training Parameters
learning_rate = 0.001
num_epochs = 1000
batch_size = 256
display_step = 100

# Network Parameters
num_classes = 5
keep_rate = 1

repeat_size = 82

if not os.path.isdir(LOG_DIR):
    os.makedirs(LOG_DIR)

In [4]:
raw = pd.read_csv('./storks_obs_train.csv', header=None)

In [5]:
mean = np.array([0. for i in  range(3)])
X = []
Y = []

for i in range(len(raw)):
    
    row = np.array(raw.loc[i][~raw.loc[i].isnull()])
    data = row[0:-1].astype(float)
    label = row[-1]
    
    x_axis = data[0::3]
    y_axis = data[1::3]
    z_axis = data[2::3]
    
    x_axis = repeat_crop_data (x_axis, repeat_size)
    y_axis = repeat_crop_data (y_axis, repeat_size)
    z_axis = repeat_crop_data (z_axis, repeat_size)
    
    mean[0] += x_axis.mean()
    mean[1] += y_axis.mean()
    mean[2] += z_axis.mean()
    
    X.append(np.stack((x_axis, y_axis, z_axis)))
    Y.append(label)

mean = mean/len(raw)
    
X = np.array(X).transpose((0, 2, 1))
Y = np.array(Y)
label_names = np.unique(Y)

le = preprocessing.LabelEncoder()
le.fit(np.unique(Y))
Y = le.transform(Y)

In [22]:
train_x, train_y, test_x, test_y, train_idx, test_idx = train_test_split(X, Y, split_rate=0.1)

In [23]:
# train_x = np.load('./storks_train_x.npy')
# train_y = np.load('./storks_train_y.npy')
# test_x = np.load('./storks_test_x.npy')
# test_y = np.load('./storks_test_y.npy')

In [24]:
class acc_dataset (torch.utils.data.Dataset):
    
    def __init__ (self, X, Y, mean):
        super().__init__()
        self.X = X
        self.Y = Y.astype(np.float)
        self.mean = mean
        
    def __len__ (self):
        return len(self.X)
    
    def __getitem__ (self, idx):
        
        x = self.X[idx]
        y = self.Y[idx]
        
        x = (x - self.mean).transpose(1, 0)
            
        return torch.tensor(x), torch.tensor(y).type(torch.LongTensor)

In [25]:
train_set = acc_dataset(train_x, train_y, mean)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = acc_dataset(test_x, test_y, mean)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [26]:
class MyNet (nn.Module):
    
    def __init__ (self, num_classes, input_dim, drop_rate):
        super().__init__()
        
        self.input_dim = input_dim
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc3 = nn.Linear(self.input_dim*64, 1024)
        self.dropout = nn.Dropout(p=drop_rate)
        self.fc4 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        #self.pretrained_resnet.avgpool(x)
        x = x.view(-1, self.input_dim*64)
        
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)

        return x

In [27]:
net = MyNet(num_classes=num_classes, input_dim=repeat_size, drop_rate=0.3).double()

net.to(device)

# Loss function
loss_function = nn.CrossEntropyLoss().double()

# Optimizer
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

# Decay LR by a factor of 0.1 every 30 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)

In [28]:
step = 0

net.train()

for epoch in range(num_epochs):
    
    exp_lr_scheduler.step()
    
    for data in train_loader:
        
        x, y = data
        
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        logits = net(x)
        
        loss = loss_function(logits, y)
        
        loss.backward()
        
        optimizer.step()
        
        step += 1
        
        _, preds = torch.max(logits, 1)
        
        correct = (preds == y).sum().item()
        
        accuracy = correct/len(x)
        
        if step % display_step == 0:
            print('Epoch: %d, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (epoch, step, loss, accuracy))
            
print('Final, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (step, loss, accuracy))


Epoch: 14, Step: 100, Minibatch loss: 0.816, Minibatch accuracy: 0.734
Epoch: 28, Step: 200, Minibatch loss: 0.670, Minibatch accuracy: 0.801
Epoch: 42, Step: 300, Minibatch loss: 0.594, Minibatch accuracy: 0.824
Epoch: 57, Step: 400, Minibatch loss: 0.546, Minibatch accuracy: 0.816
Epoch: 71, Step: 500, Minibatch loss: 0.453, Minibatch accuracy: 0.840
Epoch: 85, Step: 600, Minibatch loss: 0.375, Minibatch accuracy: 0.895
Epoch: 99, Step: 700, Minibatch loss: 0.544, Minibatch accuracy: 0.816
Epoch: 114, Step: 800, Minibatch loss: 0.344, Minibatch accuracy: 0.883
Epoch: 128, Step: 900, Minibatch loss: 0.285, Minibatch accuracy: 0.914
Epoch: 142, Step: 1000, Minibatch loss: 0.406, Minibatch accuracy: 0.863
Epoch: 157, Step: 1100, Minibatch loss: 0.272, Minibatch accuracy: 0.914
Epoch: 171, Step: 1200, Minibatch loss: 0.261, Minibatch accuracy: 0.934
Epoch: 185, Step: 1300, Minibatch loss: 0.305, Minibatch accuracy: 0.879
Epoch: 199, Step: 1400, Minibatch loss: 0.117, Minibatch accuracy: 

In [29]:
net.eval()

test_correct = 0

with torch.no_grad():
    
    for data in test_loader:

        x, y = data
        
        x, y = x.to(device), y.to(device)

        test_logits = net(x)
        _, test_preds = torch.max(test_logits, 1)

        test_correct_batch = (test_preds == y).sum().item()

        test_correct += test_correct_batch

print('Test accuracy: %.3f' % (test_correct / len(test_set)))

Test accuracy: 0.878


In [135]:
np.save('./storks_train_x.npy', train_x)
np.save('./storks_train_y.npy', train_y)
np.save('./storks_test_x.npy', test_x)
np.save('./storks_test_y.npy', test_y)

In [134]:
torch.save(net.state_dict(), os.path.join(LOG_DIR, 'model_%s_checkpoint.pth.tar' %
                                          'storks_plain_ep_1000_step_200_dr_03_acc_94_sgd'))